In [25]:
import torch
from tqdm import tqdm
import sys

sys.path.append("..")

from models.sfcn import SFCN
from utils.datasets import TorchDataset as TD

In [26]:
mode = "test"
test_block = "flat"
model_name = "PD-SFCN-GroupDRO"

In [27]:
model = SFCN(output_dim=1, channel_number=[28, 58, 128, 256, 256, 64]).to("cuda")
checkpoint = torch.load(f"checkpoints/{model_name}/best_model.pt")
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [28]:
block_dict = {"flat": model.classifier.flatten}

In [29]:
import torch
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

model.eval()

activations = []
PD = []
sex = []
study = []
scanner_type = []


def hook_fn(module, input, module_output):
    activations.append(module_output.cpu().detach().numpy())


hook = block_dict[test_block].register_forward_hook(hook_fn)

td = TD(f"/data/Data/PD/{mode}")
test_loader = DataLoader(td, batch_size=8, shuffle=False)
# Collect activations
with torch.no_grad():
    for batch in tqdm(test_loader):
        _ = model(batch[0].to("cuda"))
        PD.extend(batch[1])
        sex.extend(batch[2])
        study.extend(batch[3])
        scanner_type.extend(batch[4])
    hook.remove()

100%|██████████| 42/42 [00:08<00:00,  5.00it/s]


In [30]:
def to_onehot(labels, num_classes=None):
    # Convert to numpy array if not already
    labels = np.array(labels)

    # Determine number of classes if not provided
    if num_classes is None:
        num_classes = np.max(labels) + 1

    # Create zero matrix of shape (samples, num_classes)
    onehot = np.zeros((len(labels), num_classes))

    # Set 1s at the appropriate positions
    onehot[np.arange(len(labels)), labels] = 1

    return onehot

In [31]:
activations = np.vstack(activations).reshape(td.__len__(), -1).squeeze()
PD = np.array(PD)
sex = np.array(sex)
study = to_onehot(np.array(study))
scanner_type = to_onehot(np.array(scanner_type))

In [32]:
import os

act_save_dir = f"/data/Data/PD/{model_name}_activations_{test_block}_{mode}"
os.makedirs(act_save_dir, exist_ok=True)

for i in range(len(activations)):
    ten2 = torch.tensor(
        np.hstack([study[i], sex[i], scanner_type[i], PD[i], activations[i]])
    ).to(torch.float)
    torch.save(ten2, os.path.join(act_save_dir, f"{i}"))